### PPAML Challenge Problem 7

University of California, Berkeley

Submission for BLOG by Prof. Stuart Russell's group.

#### Flu spread model

To quickly recap: we observe region-level statistics and want to query for county-level statistic.

We will consider an *undirected model* with pairwise potentials (this is equivalent to a MV Gaussian). The potentials connect neighboring counties in space and identical counties across time (as dictated by hyperparameter $\rho$).

The primary model is built into `flu_spread_model.blog`; the primary purpose of this notebook is to perform pre-processing to get our data into BLOG correctly. We will write the following additional files:

- `flu_spread_region_rate.blog`
- `flu_spread_obs.blog`
- `flu_spread_queries.blog`

<img style="display:inline;" src="images/gmrf.png" /><img style="display:inline;"  src="images/adjacency.png" />
<img style="display:inline;" src="images/model.png" />

In [229]:
import functools
import json
import numpy as np
import pandas as pd
import re
import sys

from collections import defaultdict
from datetime import datetime

In [233]:
def is_kernel():
    if 'IPython' not in sys.modules:
        return False
    from IPython import get_ipython
    return getattr(get_ipython(), 'kernel', None) is not None

In [236]:
if not is_kernel():
    if len(sys.argv) <= 1:
        print("Need to specify training size.")
        sys.exit()
    TRAINING_SIZE = sys.argv[1]
else:
    TRAINING_SIZE = 'Middle'

#### Load the data.

Need to make sure to load from the right training data size.

In [221]:
ili_data            = pd.read_csv("data/%s/input/Flu_ILI.csv" % TRAINING_SIZE)
tweets_data         = json.load(open("data/%s/input/Flu_Vacc_Tweet_TRAIN.json" % TRAINING_SIZE))
states              = json.load(open("data/%s/input/StateInfo.json" % TRAINING_SIZE))
regions_to_counties = json.load(open("data/%s/input/Region2CountyMap.json" % TRAINING_SIZE))
county_adjacency    = json.load(open("data/%s/input/county_adjacency_lower48.json" % TRAINING_SIZE))

#### List the dates.

It's important that the dates are in chronological order.<br/>
The index of the event is important for writing observations.

In [222]:
dates = list(map(lambda s: datetime.strptime(s, "%m/%d/%Y").date().strftime('%m/%d/%Y'), ili_data["Ending"]))

In [223]:
print("Number of dates:", len(dates))

Number of dates: 103


#### Compute county statistics.

We need to compute *covariates* and *population* for each county.

$$\begin{align}
    N_c & = \texttt{ loaded from data }\\
    X_{c,t} & = \begin{bmatrix} 
                    \log{(\frac{S_{c,t} + \epsilon_2}{\tilde{N}_c})} & 
                    \log{(\frac{V_{c,t} + \epsilon_3}{1-V_{c,t}+\epsilon_3})} 
                \end{bmatrix}^T\\
\end{align}$$

Where 
$$\begin{align}
    \epsilon_2 & = 0.1\\
    \epsilon_3 & = 0.001\\
\end{align}$$

The covariate matrices should be of size $n$ by $d$.<br />
The population vector should be of size $n$ by $1$.

In [224]:
fips_to_cov1 = defaultdict(list)
fips_to_cov2 = defaultdict(list)
fips_to_pop = {}

In [225]:
for fips_code, blob in tweets_data.items():
    
    if not 'Vaccination percentage %' in blob.keys():
        continue
        
    for date in dates:
    
        if date not in blob['No. of Tweets']:
            cov1 = np.log(0.1 / blob['Population, 2014 estimate'])
            cov2 = np.log(0.001 / (1 + 0.001))
        else:
            cov1 = np.log((blob['No. of Tweets'][date] + 0.1) / blob['Population, 2014 estimate'])
            cov2 = np.log((blob['Vaccination percentage %'][date] / 100 + 0.001) / 
                          (1-blob['Vaccination percentage %'][date] / 100 + 0.001))
        
        fips_to_cov1[fips_code].append(cov1)
        fips_to_cov2[fips_code].append(cov2)
        
    fips_to_pop[fips_code] = blob['Population, 2014 estimate']

#### Construct sets of regions and counties.

We extract regions and counties for *only* the relevant counties from the training data.<br />

In [226]:
regions = set()
for i, col in enumerate(ili_data.columns):
    if i > 3:
        regions.add(col)

In [227]:
counties = set()
for r in regions:
    counties = counties.union(set(regions_to_counties[r].keys()))

In [228]:
print('Number of regions:', len(regions))
print('Number of counties:', len(counties))

Number of regions: 25
Number of counties: 277


#### Save county data.

Note: we assign an index to each county (somewhat arbitrarily).

We also create (and make sure to use) the following dictionaries:
- index_to_county
- county_to_index

In [183]:
county_to_index = {}
for i, fips in enumerate(counties):
    county_to_index[fips] = i
index_to_county = {v: k for k, v in county_to_index.items()}

In [184]:
county_pop_matrix = []
cov1_matrix = []
cov2_matrix = []

for i, fips in index_to_county.items():
    county_pop_matrix.append(fips_to_pop[fips])
    cov1_matrix.append(fips_to_cov1[fips])
    cov2_matrix.append(fips_to_cov2[fips])

county_pop_matrix = np.array(county_pop_matrix)
cov1_matrix = np.array(cov1_matrix)
cov2_matrix = np.array(cov2_matrix)
    
# np.savetxt('data_processed/county_pops.txt', county_pop_matrix)
np.savetxt('data_processed/covariates1.txt', cov1_matrix)
np.savetxt('data_processed/covariates2.txt', cov2_matrix)

In [185]:
print(cov1_matrix.shape)
print(cov2_matrix.shape)
print(county_pop_matrix.shape)

(82, 103)
(82, 103)
(82,)


#### Map regions to counties.

We construct a resulting matrix $A$ that contains
$$A_{i,j} = \begin{cases} N_j & \mbox{if region } i \mbox{ contains county } j\\
                          0 & \mbox{otherwise}  \end{cases}$$
                                               
Also calculate the region population by
$$N_r = \sum_{c \in r} N_c$$

The resulting matrix should be of size $m$ by $n$.

In [239]:
region_to_index = {}
for i, r in enumerate(regions):
    region_to_index[r] = i
index_to_region = {v: k for k, v in region_to_index.items()}

In [240]:
county_map_matrix = np.zeros((len(regions), len(counties)))
region_pop_matrix = [0] * len(regions)

for i, r in index_to_region.items():
    
    for fips in regions_to_counties[r]:
        if fips not in county_to_index:
            continue
        county_map_matrix[i][county_to_index[fips]] = county_pop_matrix[county_to_index[fips]]
        region_pop_matrix[i] += fips_to_pop[fips]

In [241]:
np.savetxt('data_processed/county_map.txt', county_map_matrix)
np.savetxt('data_processed/region_pops.txt', region_pop_matrix)

In [242]:
print(county_map_matrix.shape)

(25, 277)


#### Construct correlation matrices.

Assuming the undirected model, we have a single covariance matrix of size $n$ by $n$ (where $n$ is the number of counties). Construct following precision matrix with hyperparameter $\tau_1 \sim \mbox{Gamma}(3, 0.1)$.
$$\Sigma^{-1} = \tau_1 (D_w - W)$$

Therefore the output from this step is a matrix
$$\Sigma^{-1} = (D_w - W) + 0.01 I$$

Where $W$ is a symmetric matrix:
$$W_{i,j} = \begin{cases} 1 & \mbox{if } i \mbox{ neighbors } j\\ 0 & \mbox{otherwise} \end{cases}$$
And $D_w$ is a diagonal matrix:
$$Dw_{i,i} = \sum_j W_{i,j}$$
And $I$ is meant for regularization to ensure the matrix is positive definite.

In [190]:
W = np.zeros(((len(counties), len(counties))))

for blob in county_adjacency.values():
    
    fips = blob[1]
    neighbors = blob[2].values()
    
    if fips not in county_to_index:
        continue
        
    i = county_to_index[fips]
    for n in neighbors:
        if not n in county_to_index:
            continue
        j = county_to_index[n]
        if i == j:
            continue
        W[i][j] = 1
        W[j][i] = 1

In [191]:
np.savetxt("data_processed/W.txt", W)

In [192]:
D = np.zeros(((len(counties), len(counties))))

for i in range(len(counties)):
    D[i,i] = np.sum(W[i,:])

In [193]:
print(D)

[[ 4.  0.  0. ...,  0.  0.  0.]
 [ 0.  4.  0. ...,  0.  0.  0.]
 [ 0.  0.  6. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  5.  0.  0.]
 [ 0.  0.  0. ...,  0.  7.  0.]
 [ 0.  0.  0. ...,  0.  0.  4.]]


In [194]:
np.savetxt('data_processed/D.txt', np.diag(D))

In [195]:
# np.savetxt('data_processed/corr_cov.txt', np.eye(len(counties)))

#### Write headers for BLOG code

In [196]:
header_file = open("flu_spread_header.blog", "w")
header_file.write("""
type County;
type Region;
type Week;

distinct County counties[{0}];
distinct Region regions[{1}];
distinct Week weeks[{2}];

""".format(len(counties), len(regions), len(dates)))
header_file.close()

#### Write observations.

We ignore any entries that are NaN in the training data.

In [199]:
obs = np.zeros((len(dates), len(regions)))

In [200]:
for i, row in ili_data.iterrows():
    
    for j, region in index_to_region.items():

        if pd.isnull(row[region]):
            continue
            
        rate = float(row[region].strip('%')) / 100
        obs[i][j] = rate
#         obs_file.write('obs region_rate(regions[%d], weeks[%d]) = %0.4f;\n' % (j, i, rate))

In [201]:
np.savetxt("data_processed/obs.txt", obs)

#### Write region-level rates BLOG code, observations, and queries.

In [244]:
footer_file = open("flu_spread_footer.blog", "w")
region_variance = 0.0001

In [245]:
footer_file.write("""
random Real region_rate(Region r, Week t) ~ 
  Gaussian(
    accu(county_map[toInt(r)] * vstack(
""")
for i in range(len(counties) - 1):
    footer_file.write("      county_rate(counties[%d], t),\n" % i)
footer_file.write("      county_rate(counties[%d], t))) / region_pop[toInt(r)],\n" % (len(counties) - 1))
footer_file.write("    %f);\n\n" % region_variance)
footer_file.write("""
obs region_rate(r, t) = observations[toInt(t)][toInt(r)] for Region r, Week t : observations[toInt(t)][toInt(r)] != 0.0;
query county_rate(c, t) for County c, Week t;
""")
footer_file.close()

#### Post-processing inference output

In [11]:
!head out/output_small.txt


init time: 0.440017s

sample time: 1509.610378s (#iter = 10000000)
>> query : county_rate(County[0], Week[0])

Mean = 0.00000001   Var = 0.00000000
[0.000000, 0.050000] -> 1.00000000
(0.050000, 0.100000] -> 0.00000000
(0.100000, 0.150000] -> 0.00000000


In [12]:
import re, mmap, os

pattern = r"query : county_rate\(County\[(\d+)\], Week\[(\d+)\]\)\n{2}Mean = (\d+\.\d+)\s{3}Var = (\d+\.\d+)"
filename = 'out/output_small.txt'

f = open(filename)
size = os.stat(filename).st_size
data = mmap.mmap(f.fileno(), size, access=mmap.ACCESS_READ)

searches = re.findall(pattern, data)

In [13]:
preds = {}
for q in searches:
    preds[int(q[0]), int(q[1])] = float(q[2])

preds

{(50, 96): 0.0,
 (14, 74): 9e-08,
 (53, 53): 5e-08,
 (57, 50): 0.0,
 (21, 28): 3.155e-05,
 (4, 36): 5.06e-06,
 (78, 39): 0.0,
 (43, 3): 0.0,
 (8, 63): 0.0,
 (7, 25): 0.00160723,
 (63, 76): 8e-08,
 (48, 86): 0.0,
 (29, 44): 0.0,
 (16, 47): 0.0,
 (73, 82): 0.0,
 (1, 64): 0.0,
 (2, 78): 0.0,
 (41, 57): 0.0,
 (80, 12): 0.00468546,
 (44, 34): 3.769e-05,
 (34, 8): 3.349e-05,
 (71, 4): 2e-08,
 (56, 101): 4e-08,
 (19, 91): 0.0,
 (42, 88): 1e-08,
 (6, 98): 1.7e-07,
 (45, 61): 0.0,
 (66, 66): 0.0,
 (67, 80): 0.0,
 (13, 20): 0.00020271,
 (52, 17): 1.24e-06,
 (74, 2): 3e-08,
 (59, 97): 0.0,
 (60, 81): 5.4e-07,
 (8, 87): 0.0,
 (45, 78): 0.0,
 (50, 27): 0.00012399,
 (13, 101): 0.0,
 (53, 62): 0.0,
 (75, 97): 1.7e-07,
 (78, 46): 0.0,
 (8, 38): 0.0,
 (65, 11): 7.691e-05,
 (48, 77): 1.4e-07,
 (29, 37): 0.0,
 (68, 0): 1e-08,
 (33, 34): 0.00047889,
 (72, 67): 0.0,
 (39, 29): 1.53e-06,
 (1, 89): 0.0,
 (22, 12): 0.00020521,
 (79, 85): 1e-07,
 (26, 67): 0.0,
 (9, 9): 6.4e-07,
 (78, 8): 0.00026236,
 (12, 50)